In [1]:
import numpy as np
import pandas as pd

### Define bounds and unit transformations

In [4]:
# Define bounds of the Bay Area

lng_min, lng_max = (-123.6, -121.2)
lat_min, lat_max = (36.8, 38.9)

In [10]:
# Convert this somtime to use actual map projections
# x,y units here are kilometers

def lnglat_to_xy(lng, lat):
    x = (lng - lng_min) * 89.7
    y = (lat - lat_min) * 112.7
    return (x, y)

def xy_to_lnglat(x, y):
    lng = (x / 89.7) + lng_min
    lat = (y / 112.7) + lat_min
    return (lng, lat)

In [9]:
x_min, y_min = lnglat_to_xy(lng_min, lat_min)
x_max, y_max = lnglat_to_xy(lng_max, lat_max)

print x_min, y_min, x_max, y_max

0.0 0.0 215.28 236.67


### Load some data

In [7]:
data_path = '/Users/smmaurer/Dropbox/Data/Twitter/Westcoast-tweets-processed/'

In [5]:
%%time
tw = pd.read_hdf(data_path + 'westcoast-201510.h5', 'tweets')

CPU times: user 2.36 s, sys: 1.69 s, total: 4.05 s
Wall time: 4.54 s


In [6]:
# Limit to Bay Area

ba = tw.loc[(tw.lat > lat_min) & (tw.lat < lat_max) & 
            (tw.lng > lng_min) & (tw.lng < lng_max)].copy()

print ba.describe()

                 id            lat            lng       user_id
count  4.475370e+05  447537.000000  447537.000000  4.475370e+05
mean   6.554222e+17      37.795180    -122.226988  9.265430e+08
std    3.222303e+15       0.379636       0.453439  1.255792e+09
min    6.497705e+17      36.800205    -123.587430  2.220000e+02
25%    6.526424e+17      37.615608    -122.424600  2.572036e+07
50%    6.554710e+17      37.769765    -122.272738  1.739835e+08
75%    6.581026e+17      37.840340    -121.959000  1.589693e+09
max    6.609889e+17      38.899250    -121.200014  4.091105e+09


In [18]:
# Project to x, y coordinates

ba['x'], ba['y'] = lnglat_to_xy(ba.lng, ba.lat)

print ba[['x','y']].describe()

                   x              y
count  447537.000000  447537.000000
mean      123.159203     112.156810
std        40.673484      42.784994
min         1.127529       0.023064
25%       105.433380      91.919022
50%       119.055357     109.292502
75%       147.197736     117.246295
max       215.278740     236.585475


### Generate grid cells

In [57]:
# Define parameters

width = 0.5  # width of each cell in kilometers
overlap = 0.5  # portion of each cell that overlaps with the next, in both directions

In [58]:
%%time
# Create x, y center points

start = x_min + width/2
stop = x_max
step = width * (1 - overlap)

_x = np.arange(start, stop, step)

start = y_min + width/2
stop = y_max

_y = np.arange(start, stop, step)

# Cartesian product
_cp = np.transpose([np.tile(_x, len(_y)), np.repeat(_y, len(_x))])

gridcells = pd.DataFrame(_cp)
gridcells.columns = ['x','y']

print len(gridcells)

814506
CPU times: user 11.7 ms, sys: 11.4 ms, total: 23.1 ms
Wall time: 21.4 ms


In [59]:
%%time

# How many data points in each gridcell?

def npoints(x, y):
    '''
    Count points for gridcell with center x, y. Expects dataframe 'ba', width 'width'.
    '''
    n = len(ba.loc[(ba.x > x - width/2) & (ba.x < x + width/2) & 
                   (ba.y > y - width/2) & (ba.y < y + width/2)])    
    return n

gridcells['n'] = gridcells.apply(lambda row: npoints(row['x'], row['y']), axis=1)

print gridcells.describe()

                   x              y              n
count  814506.000000  814506.000000  814506.000000
mean      107.750000     118.375000       2.197789
std        62.137319      68.271673     121.791587
min         0.250000       0.250000       0.000000
25%        54.000000      59.250000       0.000000
50%       107.750000     118.375000       0.000000
75%       161.500000     177.500000       0.000000
max       215.250000     236.500000   54991.000000
CPU times: user 1h 3min 36s, sys: 43.4 s, total: 1h 4min 20s
Wall time: 1h 31min 31s


In [ ]:
# Should be able to speed this up using R-tree indexes...
# https://github.com/gboeing/urban-data-science/blob/master/19-Spatial-Analysis-and-Cartography/rtree-spatial-indexing.ipynb

### Save some output

In [60]:
gridcells['lng'], gridcells['lat'] = xy_to_lnglat(gridcells.x, gridcells.y)

out = gridcells[['lng','lat','n']]

out.to_csv('gridcells_all.csv')
out.loc[out.n > 500].to_csv('gridcells_top.csv')